In [ ]:
# Global Tools
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as ss
import yfinance as yf

from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
from pandas_datareader import DataReader
from scipy.stats import gaussian_kde
from scipy.integrate import simps
from scipy.optimize import minimize
from tqdm import tqdm

In [ ]:
# display all outputs of a cell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
# Data initialization
start_date = datetime(2020, 1, 1)
end_date = datetime(2024, 8, 31)
stock_symbols = ['NVDA', 'AMD', 'INTC', 'QCOM', 'AAPL', 'AVGO']
stocks= yf.download(stock_symbols, start_date, end_date)['Adj Close']

### Empirical Distribution

In [ ]:
results = pd.DataFrame(columns=['Company', 'Mean', 'Variance', 'Skewness', 'Kurtosis'])
color = ['#76B900', '#B03060', '#5A9BD5', '#002366', '#A2AAAD', '#DC143C']
plt.style.use('dark_background')
fig, ax = plt.subplots(figsize=(10, 6), dpi=150)
for i in range(len(stock_symbols)): 
    X = stocks[stock_symbols[i]].dropna().values
    mean = round(np.mean(X), 2)
    variance = round(np.var(X), 2)
    skewness = round(ss.skew(X, axis=0, bias=True), 2)
    kurtosis = round(ss.kurtosis(X, axis=0, bias=True) + 3, 2) 
    
    print(f"{stock_symbols[i]}:")
    print(f"Mean: {mean}")
    print(f"Variance: {variance}")
    print(f"Skewness: {skewness}")
    print(f"Kurtosis: {kurtosis}")
 
    sns.kdeplot(data=X, linewidth=1.5, label=stock_symbols[i], color=color[i])  

#ax.set_title("Empirical Distributions of Adjusted Close Prices (Jan 2020 - Aug 2024)", color='white')
ax.set_xlabel("Adjusted Close Price", color='grey')  
ax.set_ylabel("Density", color='white')
ax.legend(title="Company", title_fontsize='13', fontsize='11')  

### Monte-Carlo

In [1]:
def monte_carlo_simulation(mu_1, sigma1, 
                           mu_2, sigma2, 
                           p, T=1000):
    r = np.zeros(T)
    for t in range(T):
        eps1 = np.random.normal(0,1,1)
        eps2 = np.random.normal(0,1,1)
        r1 = mu_1 + sigma1 * eps1
        r2 = mu_2 + sigma2 * eps2
        u = np.random.uniform(0,1,1)
        r[t] = r1*(u <= p)+r2*(u > p)
    
    return r

def simulation_image(parameters, save_path):
    current_X, mu_1, sigma1, mu_2, sigma2, p =  parameters[0], parameters[1], parameters[2],parameters[3], parameters[4], parameters[5]
 
    plt.figure(figsize=(10,6))
    fig, ax = plt.subplots()
    sns.kdeplot(data=current_X, linewidth=4)
    initalR = monte_carlo_simulation(mu_1, sigma1, mu_2, sigma2, p, T=1000)
    plt.hist(initalR, bins=100, density=True, alpha=0.6, color='green', label="Histogram")
    ax.legend([f'Empirical Kernal Distribution: {save_path}', 'Mixture Model Simulated Distribution'])
    plt.title(f'Monte Carlo Simulation\n,sigma1={sigma1},sigma2={sigma2}')
    ## Save Image
    imagePath = os.path.join(save_path,f"{save_path},sigma1={sigma1},sigma2={sigma2}'.png")
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    plt.savefig(imagePath)

def calculateOverlap(parameters):
    
    current_X, mu_1, sigma1, mu_2, sigma2, p =  parameters[0], parameters[1], parameters[2],parameters[3], parameters[4], parameters[5]
    r = monte_carlo_simulation(mu_1, sigma1, 
                               mu_2, sigma2, 
                          p)
    
    histArea, bin_edges = np.histogram(r, bins=100, density=True)
    bin_centers = 0.5 * (bin_edges[1:] + bin_edges[:-1])
    
    
    kde = gaussian_kde(current_X)
    kdeArea = kde(bin_centers)
    
    overlap = np.minimum(histArea, kdeArea)
    return simps(overlap, bin_centers)

# Loop 

def optimize_parameters(parameters, save_path, step=5):
    bestArea = 0
    bestParam = []
    sigma1, sigma2 =  parameters[2], parameters[4]
    
    if sigma2>sigma1:
        sigma1_range = [i for i in range(sigma1-step, sigma1+step)]
        sigma2_range = [i for i in range(int(np.floor(sigma2-step*2)), sigma2+step*2)]
    
    else:
        sigma2_range = [i for i in range(sigma2-step, sigma2+step)]
        sigma1_range = [i for i in range(int(np.floor(sigma1-step*2)), sigma1+step*2)]
    
    for i in tqdm(sigma1_range):
        for j in sigma2_range:
            parameters[2] = i
            parameters[4] = j
            simulation_image(parameters, save_path)
            
            overlapArea = calculateOverlap(parameters)
            
            if bestArea <= overlapArea:
                bestParam = parameters
    return bestParam

### Monte-Carlo